In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig

import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer
import numpy as np

import pandas as pd

from sentence_transformers import SentenceTransformer

In [2]:
model_name = "google/gemma-7b-it"
# model_name = "microsoft/phi-2"


# quantization_config = BitsAndBytesConfig(load_in_8bit=False)
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map='auto')
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model.load_adapter("phi2")

streamer = TextStreamer(tokenizer=tokenizer, skip_prompt=True)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
df = pd.read_csv("public_10k_unique_rewrite_prompt.csv")

In [29]:
df

,id,original_text,rewrite_prompt,rewritten_text
0,host,"Dear Randy,\n\nI hope this letter finds you we...",Rephrase this letter to infuse it with an elfi...,"Dear Randy,\n\nMay this enchanted message find..."
1,nbroad_1,"This quilt, that my mother made, \n \n Still m...",Regency Romance: Model the text on a Regency r...,"The softest brown and brightest blue quilt, cr..."
2,nbroad_1,It's the job of our agency to keep track of th...,Write like Ernest Hemingway: Focus on Hemingwa...,The agency's responsibility is to track and co...
3,nbroad_1,"The first punch gets me right in the ribs, kno...",Grimm's Fairy Tales: Adapt the text to mimic t...,"In the sweltering sun, the stench of sweat and..."
4,nbroad_1,Some nights I lay awake staring at the ceiling...,High Fantasy Epic: Transform the essay into a ...,In the tapestry of the ethereal realm of Eldri...
...,...,...,...,...
10566,aatiffraz,"In the past 24 hours, nobody died. Not a singl...",Rewrite this text in the style of a ancient Ro...,"\n\nSure, here is the text rewritten in the st..."
10567,aatiffraz,"Oh we work today for to earn our pay, \n \n'Wa...",Restyle this text as if it were written by a h...,"\n\n## Resyled Text:\n\n""Oh, we worked today t..."
10568,aatiffraz,It's late at night and you're flipping through...,Imagine this text was a knight in the world of...,"\n\nSure, here is the rewritten text as a knig..."
10569,aatiffraz,I knew I only had 10 seconds to change things....,Rewrite this text in the style of a philosophi...,\n\n## The Knight's Tale of Temporal Flux:\n\n...


In [30]:
df["rewritten_text"].to_csv('rewritten.csv')

In [7]:
i = 0
input_text = df["rewrite_prompt"].iloc[i] + " " + df["original_text"].iloc[i]
print(input_text)

print(df["rewritten_text"].iloc[i])


'Rephrase this letter to infuse it with an elfin charm, incorporating the topic of "aid" in a whimsical and enchanting manner that captures the recipient\'s imagination and brings a sense of magic and wonder. Dear Randy,\\n\\nI hope this letter finds you well. I wanted to reach out and let you know about a new styling service we are offering at our salon, called "Trendy Styles." Our experienced stylists are skilled in creating the latest trends and can help you achieve the perfect look. Book an appointment today and let us help you elevate your style!\\n\\nBest regards,\\n[Your Name]'

'Dear Randy,\\n\\nMay this enchanted message find you in the most delightful of spirits. I simply had to share tidings of a wondrous service now gracing our humble salon, known as "Trendy Styles." Our gifted stylists possess a magical touch, conjuring the latest trends to bestow upon you the most exquisite visage. Cast your appointment spell today and allow us to weave enchantment into your style.\\n\\nWarm regards,\\n[Your Name]'

In [17]:
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(**input_ids, max_length=8196, streamer=streamer)

Styles<bos>Rephrase this letter to infuse it with an elfin charm, incorporating the topic of "aid" in a whimsical and enchanting manner that captures the recipient's imagination and brings a sense of magic and wonder. Dear Randy,\n\nI hope this letter finds you well. I wanted to reach out and let you know about a new styling service we are offering at our salon, called "Trendy Styles." Our experienced stylists are skilled in creating the latest trends and can help you achieve the perfect look. Book an appointment today and let us help you elevate your style!\n\nBest regards,\n[Your Name]

Sure, here is the rephrased letter with an elfin charm:

Dear Randy,

May this letter find you nestled beneath the moonlit canopy, your dreams dancing like fireflies. I write to you with tidings of enchantment and style, a whisper carried on the wings of a faerie.

At our salon, "Trendy Styles," we weave magic and fashion together, crafting looks that dance with the spirit of the forest. Our stylists,

In [25]:
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][len(input_text):]
print(output_text)
print(input_text)



Sure, here is the rephrased letter with an elfin charm:

Dear Randy,

May this letter find you nestled beneath the moonlit canopy, your dreams dancing like fireflies. I write to you with tidings of enchantment and style, a whisper carried on the wings of a faerie.

At our salon, "Trendy Styles," we weave magic and fashion together, crafting looks that dance with the spirit of the forest. Our stylists, like mischievous pixies, wield their shears with precision, transforming you into a vision of ethereal grace.

Book an appointment today, my dear Randy, and let us sprinkle upon you the pixie dust of fashion, elevating your style to heights never before reached.

With elfin enchantment,
[Your Name]
Rephrase this letter to infuse it with an elfin charm, incorporating the topic of "aid" in a whimsical and enchanting manner that captures the recipient's imagination and brings a sense of magic and wonder. Dear Randy,\n\nI hope this letter finds you well. I wanted to reach out and let you kn

In [21]:
from sentence_transformers import SentenceTransformer
sentenceTF = SentenceTransformer('sentence-transformers/sentence-t5-base')

In [23]:
def sim(s, k, p=3):
    sign = np.sign(np.dot(s, k))
    s_norm = np.linalg.norm(s)
    k_norm = np.linalg.norm(k)

    abs_dot_product = np.abs(np.dot(s, k)) ** p
    result = sign * (abs_dot_product / (s_norm * k_norm))
    return result

def get_embeddings(text):
    return sentenceTF.encode(text)

def get_sim(text1, text2):
    embeddings = sentenceTF.encode([text1, text2])
    return sim(embeddings[0], embeddings[1])

In [26]:
get_sim(input_text, output_text)

0.8814387612300727